In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import mahalanobis

In [3]:
df = pd.read_csv('../data/allinone.csv')

### ELIMINAR COLUMNAS 

In [4]:
# Eliminar columnas con un solo valor único
df = df.loc[:, df.nunique() > 1]

In [5]:
# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

In [6]:
# Eliminar columnas no relevantes
df = df.drop(columns=['Unnamed: 0'])

### Manejo de Valores Nulos

In [7]:
# Comprobamos la cantidad de nulos en las columnas
df.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 69, dtype: int64

In [8]:
# Eliminamos las columnas con >50% de valores nulos 

umbral = len(df) * 0.5 
df = df.dropna(thresh=umbral, axis=1) 

In [9]:
# Verificar valores infinitos en columnas numéricas
inf_check = np.isinf(df.select_dtypes(include=['int64', 'float64'])).sum()
print("Valores infinitos por columna:\n", inf_check)

Valores infinitos por columna:
  Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
 Active Min                    0
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
Length: 68, dtype: int64


In [10]:
df = df.replace([np.inf, -np.inf], np.nan)  # Reemplazar inf con NaN

In [11]:
# Imputación para columnas numéricas (todas las int y float)
col_num = df.select_dtypes(include=['int64', 'float64']).columns
imputer_num = SimpleImputer(strategy='mean')
df[col_num] = imputer_num.fit_transform(df[col_num])

In [12]:
# Imputación para columnas categóricas 
col_cat = df.select_dtypes(include=['object']).columns
imputer_cat = SimpleImputer(strategy='most_frequent')
df[col_cat] = imputer_cat.fit_transform(df[col_cat])

In [13]:
# Eliminar filas con nulos residuales 

df = df.dropna()

###  CORREGIR INCONSISTENCIAS 

In [14]:
# Estandarizar formatos en columnas categóricas (texto en minúsculas) 
for i in col_cat:
    df[i] = df[i].str.lower().str.strip()

In [15]:
# Eliminar duplicados 

df=df.drop_duplicates() 

### MANEJO DE OUTLIERS 

In [16]:
# Método IQR con cuantiles para la detección de outliers (no se detectan outliers)
Q1 = df[col_num].quantile(0.25)
Q3 = df[col_num].quantile(0.75)
IQR = Q3 - Q1

# Filtramos las filas cuyos valores estén dentro de los límites
df_filtrado = df[(df[col_num] >= (Q1 - 1.5 * IQR)) & (df[col_num] <= (Q3 + 1.5 * IQR))]

print("Filas originales:", len(df))
print("Filas después del filtro IQR:", len(df_filtrado))

Filas originales: 2282168
Filas después del filtro IQR: 2282168


In [17]:
# Método de la Desviación Absoluta Mediana (MAD) (no se detectan outliers)
median = df[col_num].median()
mad = np.median(np.abs(df[col_num] - median))

# Verificar si MAD es cero para evitar división por cero
if mad == 0:
    print("MAD es 0; revisar la variabilidad de la columna")
else:
    z_score = 0.6745 * (df[col_num] - median) / mad
    
    # Filtrar las filas que no son outliers
    df_filtrado = df[(np.abs(z_score) <= 3.5)]
    
    print("Filas originales:", len(df))
    print("Filas después del filtro MAD:", len(df_filtrado))

Filas originales: 2282168
Filas después del filtro MAD: 2282168


#### Distancia de Mahalanobis

La distancia de Mahalanobis es una medida que permite calcular la distancia entre un punto y una distribución, teniendo en cuenta las correlaciones entre variables en un conjunto de datos multivariado. A diferencia de la distancia euclidiana, que trata a todas las variables por igual, la distancia de Mahalanobis ajusta las distancias según la varianza y covarianza de los datos, lo que la hace especialmente útil para detectar outliers en datos con múltiples variables interrelacionadas.

In [ ]:
# Método con la distancia de Mahalanobis (no se encuentran outliers)
col_num = df.select_dtypes(include=[np.number]).columns
data = df[col_num].values

# Calcular la media, la matriz de covarianza y la matriz pseudo-inversa en lugar de la inversa normal para evitar un problema de singular matrix que tenia antes
mean = np.mean(data, axis=0)
cov_matrix = np.cov(data, rowvar=False)
inv_cov_matrix = np.linalg.pinv(cov_matrix)

# Calcular la distancia de Mahalanobis para cada punto
mahalanobis_distances = [mahalanobis(row, mean, inv_cov_matrix) for row in data]

# Determinar un umbral para identificar outliers
threshold = np.percentile(mahalanobis_distances, 97.5)  # Por ejemplo, el percentil 97.5

# Identificar los outliers
outliers = df[mahalanobis_distances > threshold]

print("Outliers encontrados: ", len(outliers))

Outliers encontrados:  57055


In [21]:
outliers

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
20,53528.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
75,34805.0,1.0,2.0,0.0,617.0,0.0,617.0,0.0,308.500000,436.284884,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
80,80.0,60130920.0,5.0,3.0,216.0,1533.0,198.0,0.0,43.200000,86.574823,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
81,80.0,60130912.0,5.0,3.0,213.0,1619.0,195.0,0.0,42.600000,85.233796,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
90,80.0,60166984.0,6.0,4.0,237.0,4593.0,213.0,0.0,39.500000,85.031171,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071523,52330.0,1.0,1.0,1.0,6.0,6.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
3071561,49788.0,28090.0,5.0,1.0,5494.0,6.0,1460.0,6.0,1098.800000,629.621474,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
3071581,443.0,59077.0,3.0,1.0,18.0,1448.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,benign
3071621,3268.0,115491871.0,22.0,13.0,2986.0,6356.0,1305.0,0.0,135.727273,379.293351,...,32.0,16944.0,0.0,16944.0,16944.0,115000000.0,0.0,115000000.0,115000000.0,benign


In [23]:
# Eliminamos los outliers
df = df[~df.index.isin(outliers.index)]

### NORMALIZACIÓN/ESTANDARIZACIÓN DE LOS DATOS 

In [ ]:
# Estandarización (media = 0 desviación = 1) 

# scaler = StandardScaler() 
# df[col_num] = scaler.fit_transform(df[col_num]) 

### CODIFICACIÓN DE VARIABLES CATEGÓRICAS 

In [ ]:
# Identificar primero variables categóricas 

# print(df.dtypes) 

# for col in df.select_dtypes(include=['object']).columns: 
#     		print(f"{col}: {df[col].nunique()} valores únicos") 

In [ ]:
# Codificación ordinal para categrorías con orden con el uso de LabelEncoder de la librería de Sci-kit learn
# for i in col_cat:
#     le = LabelEncoder()
#     df[i] = le.fit_transform(df[i]) 

In [ ]:
# One-Hot Enccoding para categorías sin orden 

# df = pd.get_dummies(df, columns=col_cat, drop_first=True, dummy_na=True)

### FEATURE ENGINEERING

In [24]:
# TOTAL PAQUETES
df['Total_Packets'] = df[' Total Fwd Packets'] + df[' Total Backward Packets']

In [25]:
# Ratio de paquetes enviados/recibidos para identificar desequilibrios en la comunicación
df['Fwd_Bwd_Packet_Ratio'] = df[' Total Fwd Packets'] / (df[' Total Backward Packets'] + 1)

In [26]:
df['Duration_per_Packet'] = df[' Flow Duration'] / df['Fwd_Bwd_Packet_Ratio']

In [27]:
# Tasa de bytes por segundo
df['Bytes_per_Second'] = (df['Total Length of Fwd Packets'] + df[' Total Length of Bwd Packets']) / (df[' Flow Duration'] / 1e6 + 1)

In [28]:
# Numero total de flags activos
df['Total_Active_Flags'] = (df['FIN Flag Count'] + df[' SYN Flag Count'] + df[' RST Flag Count'] + df[' PSH Flag Count'] + df[' ACK Flag Count'] + df[' URG Flag Count'] + df[' ECE Flag Count'])

### GUARDAR DATOS LIMPIOS EN CSV 

In [29]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Total_Packets,Fwd_Bwd_Packet_Ratio,Duration_per_Packet,Bytes_per_Second,Total_Active_Flags
count,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,...,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06,2.225113e+06
mean,8.573467e+03,1.595934e+07,5.449007e+00,5.218862e+00,4.326432e+02,4.617155e+03,2.000028e+02,1.694799e+01,5.389739e+01,6.636747e+01,...,4.460925e+04,9.474002e+06,4.116477e+05,9.760677e+06,9.146908e+06,1.066787e+01,1.057761e+00,1.713226e+07,1.048411e+03,7.787375e-01
std,1.896537e+04,3.466052e+07,1.463069e+01,2.055569e+01,1.535186e+03,3.555565e+04,5.664579e+02,2.281437e+01,1.450839e+02,2.200529e+02,...,3.431626e+05,2.537371e+07,4.282186e+06,2.592374e+07,2.518119e+07,3.489807e+01,1.618077e+00,3.835476e+07,8.222037e+03,6.835205e-01
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,2.272727e-02,-2.600000e+01,0.000000e+00,0.000000e+00
25%,5.300000e+01,2.150000e+02,1.000000e+00,1.000000e+00,1.800000e+01,6.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,5.000000e-01,3.015000e+02,1.199939e+01,0.000000e+00
50%,8.000000e+01,5.096300e+04,2.000000e+00,2.000000e+00,6.600000e+01,1.620000e+02,4.000000e+01,2.000000e+00,3.700000e+01,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,6.666667e-01,7.691700e+04,1.386490e+02,1.000000e+00
75%,4.430000e+02,5.204591e+06,5.000000e+00,5.000000e+00,3.270000e+02,1.471000e+03,2.010000e+02,3.700000e+01,5.200000e+01,7.499905e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01,1.176471e+00,3.489621e+06,3.079350e+02,1.000000e+00
max,6.553500e+04,1.200000e+08,3.568000e+03,4.733000e+03,5.815500e+04,9.947167e+06,8.760000e+03,9.170000e+02,2.326400e+03,3.803230e+03,...,9.418959e+06,1.200000e+08,6.430000e+07,1.200000e+08,1.200000e+08,8.301000e+03,2.990000e+02,3.476377e+08,2.295654e+06,3.000000e+00


In [30]:
df.shape

(2225113, 74)

In [31]:
df.to_csv('../data/datos_limpios.csv')